# Objetivo: Note base que implementa funciones para script procesador de productos
# Autor: Gustavo V. Diaz
# Fecha: 22/09/2024

### Descripción
Script base que servirá para copiar y pegar esctructura general y funciones desarrolladas en módulo especializado para las tareas necesarias.

Se parte de la hipótesis de tener definido un dataframe que contenga la búsqueda realizada por "searcher_filt.py" junto con el kml de búsqueda inicial. Posteriormente se irá recorriendo el dataframe para:
1. Bajar producto
2. Recortarlo
3. Ingresar polígono de búsqueda a producto SNAP
4. Aplicar resize a producto SNAP
5. Enmascarar cirros
6. Enmascarar por ROI de búsqueda
7. Procesar por NDVI o RGB
8. Entregar salida según se requiera en el paso 7

#### Entrada
1. df de productos a bajar (en csv o pkl)
2. Ruta kml de búsqueda

#### Salida

1. Carpeta de salida
    1. Composiciones RGB
        1. Tiff
        1. png
    2. Tabla metadatos
        1. Productos
            1. Id
            2. acq_date
            3. Tiff_Prod_name (salida)
            4. Ruta a Tiff_Prod_name
        2. Configuración
            1. Configuración de búsqueda
            2. Cantidad de productos procesados
            3. Fecha inicio análisis
            4. Fecha fin análisis
            5. Fecha procesamiento
            6. Nombre ROI

In [1]:
# Declaración de módulos necesarios
# Para lectura de csv
import pandas as pd

# Declaración de variables para configuración
input_path = r'/src/output/research_20240928T190500_Tratayen.csv'
input_path_pkl = r'/src/output/research_20240928T190500_Tratayen.pkl'
verbose = True
kml_path = r'/src/Vectores/Tratayen.kml'

In [2]:
# Lectura de dataframes de búsqueda a partir del csv

def lectura_csv(path, verbose):
    df = pd.read_csv(path)
    if verbose:
        print(f'Muestro variable path de función {lectura_csv.__name__}')
        print()
        display(df)
    return df

def lectura_pkl(path, verbose):
    df = pd.read_pickle(path)
    if verbose:
        print(f'Muestro variable path de función {lectura_csv.__name__}')
        print()
        display(df)
    return df

In [3]:
df = lectura_csv(input_path, verbose)

Muestro variable path de función lectura_csv



,Unnamed: 0,Unnamed: 1,Unnamed: 2,cloudCover,Id,Name,shape,acq_date
0,MSIL2A,T19HET,N0509,14.980999,33a0f98c-5167-5a72-9fb9-0ed7fbb231bc,S2B_MSIL2A_20230107T141709_N0509_R010_T19HET_2...,"POLYGON ((-69.00021 -37.9475895719933, -69.000...",2023-01-07 14:17:09
1,MSIL2A,T19HET,N0509,1.562553,ea3043f6-abea-5cb3-ad34-608c33992e76,S2A_MSIL2A_20230112T141711_N0509_R010_T19HET_2...,"POLYGON ((-69.00021 -37.9475895719933, -69.000...",2023-01-12 14:17:11
2,MSIL2A,T19HET,N0509,0.000562,64994d5b-4b3f-5d57-bbf0-e3ef099338e7,S2A_MSIL2A_20230115T142711_N0509_R053_T19HET_2...,"POLYGON ((-68.2912 -37.9438257932026, -69.0002...",2023-01-15 14:27:11
3,MSIL2A,T19HET,N0509,1.742787,dad956c6-a9f9-521f-b5d9-bb01f969b777,S2B_MSIL2A_20230120T142709_N0509_R053_T19HET_2...,"POLYGON ((-68.287964 -37.9438086140647, -69.00...",2023-01-20 14:27:09
4,MSIL2A,T19HET,N0509,0.401804,10e5880c-bef8-54d1-8464-90ad7c24cef8,S2A_MSIL2A_20230122T141711_N0509_R010_T19HET_2...,"POLYGON ((-69.00021 -37.9475895719933, -69.000...",2023-01-22 14:17:11
5,MSIL2A,T19HET,N0509,0.001960,735dd08b-3710-5a9e-ac60-bbfdaa484afd,S2A_MSIL2A_20230125T142711_N0509_R053_T19HET_2...,"POLYGON ((-68.298065 -37.9438622103473, -69.00...",2023-01-25 14:27:11
6,MSIL2A,T19HET,N0509,0.001072,cc1aefa8-45f9-52fd-bdfe-92c20b083c46,S2B_MSIL2A_20230127T141709_N0509_R010_T19HET_2...,"POLYGON ((-69.0002276354548 -37.9475895719933,...",2023-01-27 14:17:09
7,MSIL2A,T19HET,N0509,0.005158,5f35ecac-f932-51d6-84f4-ea149c5514be,S2B_MSIL2A_20230130T142709_N0509_R053_T19HET_2...,"POLYGON ((-68.28497 -37.9437926006053, -69.000...",2023-01-30 14:27:09
8,MSIL2A,T19HET,N0509,0.002573,5bf9fff7-ca08-58b6-a99a-fdbe17e15998,S2A_MSIL2A_20230204T142711_N0509_R053_T19HET_2...,"POLYGON ((-68.30292 -37.9438879926409, -69.000...",2023-02-04 14:27:11
9,MSIL2A,T19HET,N0509,16.354923,3e2eaa5c-9a83-5a7d-94fc-00aea0bc41b2,S2B_MSIL2A_20230206T141709_N0509_R010_T19HET_2...,"POLYGON ((-69.00021 -37.9475895719933, -69.000...",2023-02-06 14:17:09


In [4]:
# Prueba para lectura de df en pkl, no pude leerla
# lectura_pkl(input_path_pkl, verbose)
# Error: ValueError: unsupported pickle protocol: 5

In [10]:
# Iteración sobre todos los productos
# Dejo una configuración que me permitirá solo procesar el primer producto,\
# cuando verifique que funciona dehabilitaré la función para que procese todos los productos de la misma manera que el primero
flag_one_proc = True
for row in df.iterrows():
    print(row[1]['Id'])
    print(row[1]['Name'] + '\n')
    
    if flag_one_proc == True:
        break
    

33a0f98c-5167-5a72-9fb9-0ed7fbb231bc
S2B_MSIL2A_20230107T141709_N0509_R010_T19HET_20230107T181706.SAFE

ea3043f6-abea-5cb3-ad34-608c33992e76
S2A_MSIL2A_20230112T141711_N0509_R010_T19HET_20230112T210053.SAFE

64994d5b-4b3f-5d57-bbf0-e3ef099338e7
S2A_MSIL2A_20230115T142711_N0509_R053_T19HET_20230115T191054.SAFE

dad956c6-a9f9-521f-b5d9-bb01f969b777
S2B_MSIL2A_20230120T142709_N0509_R053_T19HET_20230120T184034.SAFE

10e5880c-bef8-54d1-8464-90ad7c24cef8
S2A_MSIL2A_20230122T141711_N0509_R010_T19HET_20230122T210406.SAFE

735dd08b-3710-5a9e-ac60-bbfdaa484afd
S2A_MSIL2A_20230125T142711_N0509_R053_T19HET_20230125T190652.SAFE

cc1aefa8-45f9-52fd-bdfe-92c20b083c46
S2B_MSIL2A_20230127T141709_N0509_R010_T19HET_20230127T182917.SAFE

5f35ecac-f932-51d6-84f4-ea149c5514be
S2B_MSIL2A_20230130T142709_N0509_R053_T19HET_20230130T184428.SAFE

5bf9fff7-ca08-58b6-a99a-fdbe17e15998
S2A_MSIL2A_20230204T142711_N0509_R053_T19HET_20230204T211800.SAFE

3e2eaa5c-9a83-5a7d-94fc-00aea0bc41b2
S2B_MSIL2A_20230206T141709_